In [1]:
import subprocess
import itertools
import random
from random import randrange
import statistics 

In [2]:
STUDENT_ID = 12021446

def check(student_id , passwords) :
    proc = subprocess.Popen(["./unlock64", str(student_id)] + passwords , stdout = subprocess.PIPE)
    results = []
    while True :
        line = proc.stdout.readline()
        if not line :
            break
        results.append(float(str(line).split("\\t")[1].split("\\r\\n")[0]))
    return results

print(check(STUDENT_ID , ["PASSWORD", "ALGOGEN"]))

[0.328869, 0.344896]


In [3]:
GENES = "ABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890"

In [4]:
def generationIndividu(size):
    return [random.choice(GENES) for i in range(size)]

In [5]:
print(generationIndividu(18))

['Z', 'E', '0', '2', 'Z', 'E', '0', 'T', 'S', 'L', 'M', 'N', 'C', 'N', 'Z', 'I', 'B', '8']


In [6]:
#Le nombre d'individus à générer est passé en paramètre
def generationPopulation(nombreIndividus):
    return [generationIndividu(randrange(12,19)) for i in range(nombreIndividus)]

In [7]:
population = generationPopulation(10)

In [8]:
"""
    Mutation d'un gène
"""
def mutation(enfants, prop_mutee = 1/ 100):
    for enfant in enfants :
        if random.random() < prop_mutee:
            child1 = toStr(enfant)
            indexGene = randrange(len(enfant))
            enfant[indexGene] = random.choice(GENES)
            child2 = toStr(enfant)
            #print("mutation : ", child1, child2)
    #print("end mutation")
    """ 
    for enfant in enfants :
        print(toStr(enfant))
    """
    return enfants


"""
    Ajout ou suppression aléatoire d'un géne
    On s'assure que l'on a bien entre 12 et 18 genes
"""
def addOrRemoveGene(enfants, prob_add_gene = 1/100, prob_remove_gene = 1/100):
    for enfant in enfants :
        if(len(enfant)<18):
            if(random.random()<prob_add_gene):
                # Ajouter un gene
                gene = random.choice(GENES)
                indexGene = randrange(len(enfant))
                enfant.insert(indexGene, gene)
            if(random.random()<prob_remove_gene):
                # Retirer un gene
                indexGene = randrange(len(enfant))
                enfant.remove(random.choice(enfant))
    return enfants



In [9]:
#Croisement des parents pour créer une nouvelle génération
def croisement( population, parents, nombreEnfantsSouhaites):
    """ 
    nombreDeParentsSelectionnes = len(parents)
    nombreEnfantsSouhaites = len(population)-nombreDeParentsSelectionnes
    nombreEnfantsSouhaites = NB_INDIVIDUS
    """
    probAddGene = 0.2
    probRemoveGene = 0.2
    enfants = []
    while len(enfants) < nombreEnfantsSouhaites:
        pere = random.choice(parents)
        mere = random.choice(parents)
        #Le nombre de gènes du père sera égal à la longueur du code secret divisé par deux
        nombreGenesPere = len(pere)//2
        #Le nombre de gènes de la mère sera égal à la longueur du mot - le nombre de gène du père
        nombreGenesMere = len(mere)//2
        #print("nb gene :",nombreGenesPere +nombreGenesMere)        
        enfant = pere[:nombreGenesPere] + mere[nombreGenesMere:]
        """ 
        if(len(enfant)<18):
            if(random.random()<probAddGene):
                # Ajouter un gene
                gene = random.choice(GENES)
                indexGene = randrange(len(enfant))
                enfant.insert(indexGene, gene)
        if(random.random()<probRemoveGene):
                # Retirer un gene
                indexGene = randrange(len(enfant))
                enfant.remove(random.choice(enfant))
            #enfant.remve(gene)
        """
        # Ajouter l'enfant
        enfants.append(enfant)  
    parents.extend(enfants)
    return enfants





In [10]:
def evaluation(population):    
    listPasswords = []
    for individu in population:
        nextPass = "".join(individu)
        listPasswords.append(nextPass)
    #print("evaluation nb pass:", len(listPasswords))
    result = check(STUDENT_ID , listPasswords)
    #print("evaluation", result)
    idx = 0
    list_evaluation = []
    for individu in population:
        list_evaluation.append((individu, result[idx]))
        idx+=1
    # Trier les tuples(élement,score) par score décroissant
    list_evaluation2 = sorted(list_evaluation, key=lambda tuple:-1*tuple[1])
    #print("evaluation__2", list_evaluation2)
    return list_evaluation2


In [11]:





def tirer_individu(list_evaluation, individus_score, individus_score_rang):
    #print("tirer_individu", individus_score)
    sumScore = sum(individus_score)
    sumScoreRang = sum(individus_score_rang)
    individus_proba = []
    #print("tirer_individu", individus_score, sumScore)
    """ 
    for score in individus_score:
        proba = (score/sumScore)
        #print("tirer_individu proba", proba)
        #proba = (score/sumScore)
        individus_proba.append(proba)
    """
    for score_rang in individus_score_rang:
        proba = (score_rang/sumScoreRang)
        #print("tirer_individu proba", proba)
        #proba = (score/sumScore)
        individus_proba.append(proba)
    #print("tirer_individu proba :", individus_proba, sum(individus_proba))
    random1 = random.random()    
    sumProba = 0
    index = 0
    for proba in individus_proba:
        sumProba = sumProba + proba
        if(sumProba > random1 ):
            #print("tirer_individu : return idx", index)
            return list_evaluation[index]
        index+=1
    #print("tirer_individu : return idx", index-1)
    return (list_evaluation[index-1])


def selection(list_evaluation, pourcentage_selec, NB_INDIVIDUS):
    individus_evaluations = []
    individus_score = []    
    individus_score_rang = []
    selected_elts = []
    nb_parent = int(len(individus_evaluations)*pourcentage_selec)
    index = 0
    for individu, score in list_evaluation :
        #if score >= 0.9 :
        #    return individu
        individus_evaluations.append(individu)
        individus_score.append(score)
        individus_score_rang.append(NB_INDIVIDUS- index)
        if(index<nb_parent):
            selected_elts.add(individu)
        index = index+1
    #print("slection list_evaluation", list_evaluation)
    #Nombre des parents à garderevaluation__
    """
    parents = individus_evaluations[:nb_parent]
    #print("selection step1", len( parents))
    proba = 0
    index = nb_parent
    for individu in individus_evaluations[nb_parent:]:
        proba = proba + (individus_score[index]/sum(individus_score))
        if random.random() < proba:
            parents.append(individu)
        index = index +1 
    #print("selection parents1 ", len(parents), parents)
    return parents
    """
    evaluation
    selected_elts = []
    #nb_parent = len(list_evaluation)/2
    nb_parent2 = NB_INDIVIDUS - nb_parent
    for x in range(nb_parent2):
        toAdd, score =  tirer_individu(list_evaluation, individus_score, individus_score_rang)
        #print("tirer_individu returned", toStr2(toAdd, score))
        selected_elts.append(toAdd)
        #print("selection", toAdd, selected_elts)
    
    #print("selection parents2 ", len(selected_elts), selected_elts)
    return selected_elts


def toStr(individu):
    length = len(individu)
    return "".join(individu) + " [" + str(length) + "]"

def toStr2(individu, score):
    return toStr(individu)+ ":" + str(score)


def log_list_evaluation(list_evaluation):
    print("### log_population")
    #print('log_population individus_score', list_evaluation)
    idx=0
    for individu,score in list_evaluation:
        #print("log_population : individus_score",individu )
        print(toStr2(individu,score))


    

In [12]:
NB_INDIVIDUS = 100
NB_INDIVIDUS = 30
PROB_MUTATION = 0.7
PROB_ADD_GENE = 0.2
PROB_REMOVE_GENE = 0.2
MAX_GENERATIONS = 10000
POURCENTAGE_SELECTION = 0.2

population = generationPopulation(NB_INDIVIDUS)
i = 0
score = 0
fitness = []
nb_to_keep = int(NB_INDIVIDUS*POURCENTAGE_SELECTION)
nb_children = NB_INDIVIDUS - nb_to_keep

print("------ Begin nb_to_keep = ", nb_to_keep , " nb_children to produce = " , nb_children)

while i < MAX_GENERATIONS and score <= 0.99 :    
    list_evaluation = evaluation(population)
    individus_score = []
    for individu, score in list_evaluation :
        individus_score.append(score)
    if(i%10==0):
        log_list_evaluation(list_evaluation)
    fitness.append((max(individus_score), statistics.mean(individus_score)))
    #print("population:", len(individus_score))
    print("iter", i ,"score max: ",fitness[i][0], "score moyen: ",fitness[i][1])
    parents = selection(list_evaluation, POURCENTAGE_SELECTION, NB_INDIVIDUS)
    #print("step1, parent size", len(parents))
    enfants = croisement(population, parents, nb_children)
    #print("step2, enfants size", len(enfants))
    enfants = mutation(enfants, PROB_MUTATION)    
    #print("step3, enfants size", len(enfants))
    enfants = addOrRemoveGene(enfants, PROB_ADD_GENE, PROB_REMOVE_GENE)
    population= []
    # Garder ceux qui ont obetnu les meilleurs score (on ne les modifie pas)
    index = 0
    #print(list_evaluation)
    first_elts = list_evaluation[:nb_to_keep]    
    for individu, score in first_elts:
        #print("first_elts", individu, score)
        population.append(individu)
 
    #print("step4, population size", len(population))
    #population = enfants
    population.extend(enfants)        
    i = i + 1

------ Begin nb_to_keep =  6  nb_children to produce =  24
### log_population
7X79DNOCJN8F [12]:0.584279
58WYPX5EQ4EFN [13]:0.561757
OFPM5QWOV2ZD [12]:0.537679
ZWABK5C6HZVDMHAW5 [17]:0.537075
XLTXCJU4JKF3EOS [15]:0.536681
SDK12PJDK7H3TTM43 [17]:0.521943
O2656VDBWHIZWFCF [16]:0.515309
4L9D813Y4YBJS [13]:0.514611
78S6MM36L613 [12]:0.514053
071AOR5NTZWY69V [15]:0.50607
EC2UGGW7K7C3 [12]:0.499571
L41NT6P8HB9KIN5 [15]:0.493786
F7TZ5RM2W5JSNB75K [17]:0.48957
S36J1XV1OVIR [12]:0.488538
93O72LLMBC9LXZ80Z [17]:0.481813
6PY8JR2L3QRZ [12]:0.476121
JEVYZD2HW5XI [12]:0.476121
H3D9BGVD5M06Y [13]:0.46847
P8LUCAEK8JUYMO0CSY [18]:0.467931
MY2SSCHBOIF6F [13]:0.462083
IGOCC1DEZRRVPG24QT [18]:0.461208
3XXZJVBJJDLQS [13]:0.454567
V2WZ3ID6TUFYA2 [14]:0.449498
N3HJMX8IF7XZ [12]:0.448563
GUMVM1O454GAZLEC [16]:0.431283
NHINS52OAK674Q [14]:0.422312
BM6MNAN1NXM0V447 [16]:0.407441
UJCZTYOKOYVXH [13]:0.392214
QW6QJSQG26REIAJ4XI [18]:0.369894
PJ05YHDLIJIO2BZJM [17]:0.368082
iter 0 score max:  0.584279 score moyen: 

KeyboardInterrupt: 

### 